# Ablation V0: LoRA Minimal Baseline (r=16, alpha=32)

This notebook is part of an ablation study to compare different fine-tuning approaches.

## Model and Approach
- **Base Model**: `facebook/mms-tts-amh` (VITS architecture, ~36.3M parameters)
- **Fine-tuning Method**: LoRA (r=16, alpha=32)
- **Task**: Text-to-Speech (TTS)
- **Domain**: Legal Amharic text
- **Dataset**: Dataset_4.0h
- **Text Format**: Romanized Amharic (using uroman package)


## 1. Installation and Setup


In [ ]:
%pip install -q transformers datasets accelerate peft torchaudio librosa soundfile uroman scipy matplotlib


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import librosa
import soundfile as sf
import scipy.io.wavfile
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from transformers import (
 VitsModel,
 AutoTokenizer,
 TrainingArguments,
 Trainer
)

from peft import (
 LoraConfig,
 get_peft_model,
 TaskType
)

from datasets import Dataset, DatasetDict

# Import uroman for romanization
import uroman

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
 print(f"CUDA device: {torch.cuda.get_device_name(0)}")
 print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import librosa
import soundfile as sf
import scipy.io.wavfile
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from transformers import (
    VitsModel,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType
)

from datasets import Dataset, DatasetDict

# Import uroman for romanization
import uroman

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 2. Configuration


In [ ]:
MODEL_NAME = "facebook/mms-tts-amh"

# Dataset paths - using Dataset_4.0h
AUDIO_DIR = "/content/drive/MyDrive/Dataset_4.0h/audio"
TRAIN_CSV = "/content/drive/MyDrive/Dataset_4.0h/train.csv"
VAL_CSV = "/content/drive/MyDrive/Dataset_4.0h/val.csv"
TEST_CSV = "/content/drive/MyDrive/Dataset_4.0h/test.csv"

OUTPUT_DIR = "mms_tts_lora_v0"

# LoRA Configuration
LORA_CONFIG = {
 "r": 16,
 "lora_alpha": 32,
 "target_modules": ["q_proj", "v_proj", "k_proj", "out_proj"],
 "lora_dropout": 0.1,
 "bias": "none",
 "task_type": "FEATURE_EXTRACTION"
}

# Training configuration
TRAINING_ARGS = {
 "output_dir": OUTPUT_DIR,
 "per_device_train_batch_size": 2, # Reduced for T4 GPU
 "per_device_eval_batch_size": 2,
 "gradient_accumulation_steps": 8, # Effective batch size 16
 "learning_rate": 5e-5,
 "warmup_steps": 100,
 "max_steps": 3000, # Increased for larger dataset
 "gradient_checkpointing": True,
 "fp16": True,
 "eval_strategy": "steps",
 "eval_steps": 500,
 "save_strategy": "steps",
 "save_steps": 500,
 "save_total_limit": 3,
 "load_best_model_at_end": True,
 "metric_for_best_model": "loss",
 "greater_is_better": False,
 "logging_steps": 100,
 "report_to": "none",
 "push_to_hub": False
}

print("Configuration:")
print(f" Model: {MODEL_NAME}")
print(f" Output directory: {OUTPUT_DIR}")
print(f" LoRA rank (r): {LORA_CONFIG['r']}")
print(f" LoRA alpha: {LORA_CONFIG['lora_alpha']}")


## 3. Load and Prepare Data


In [ ]:
def load_csv_split(csv_path, audio_dir):
    """Load a CSV split and return list of (audio_path, transcription) tuples"""
    df = pd.read_csv(csv_path)

    data = []
    for _, row in df.iterrows():
        audio_path = Path(audio_dir) / row['file_name']
        transcription = str(row['transcription']).strip()

        if audio_path.exists():
            data.append({
                'audio_path': str(audio_path),
                'transcription': transcription
            })
        else:
            print(f"Warning: Audio file not found: {audio_path}")

    return data

train_data = load_csv_split(TRAIN_CSV, AUDIO_DIR)
val_data = load_csv_split(VAL_CSV, AUDIO_DIR)
test_data = load_csv_split(TEST_CSV, AUDIO_DIR)

print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")
print(f"\nTotal samples: {len(train_data) + len(val_data) + len(test_data)}")




## 4. Romanize Text Using Uroman


In [ ]:
# Initialize uroman instance
try:
    uroman_obj = uroman.Uroman()
    print("Uroman initialized successfully")
except Exception as e:
    print(f"Error initializing uroman: {e}")
    uroman_obj = None

def romanize_text(text):
    """Convert Ge'ez script Amharic text to Romanized format using uroman"""
    if uroman_obj is None:
        return text # Return original if uroman not available

    try:
        # Uroman.romanize_string() is the correct method
        romanized = uroman_obj.romanize_string(text)
        return romanized.strip()
    except Exception as e:
        print(f"Error romanizing text: {text[:50]}... Error: {e}")
        return text # Fallback to original if romanization fails

# Test romanization on a sample
sample_text = train_data[0]['transcription']
print(f"Original (Ge'ez): {sample_text}")
romanized_sample = romanize_text(sample_text)
print(f"Romanized: {romanized_sample}")




In [ ]:
# Initialize uroman instance
try:
    uroman_obj = uroman.Uroman()
    print("Uroman initialized successfully")
except Exception as e:
    print(f"Error initializing uroman: {e}")
    uroman_obj = None

def romanize_text(text):
    """Convert Ge'ez script Amharic text to Romanized format using uroman"""
    if uroman_obj is None:
        return text # Return original if uroman not available

    try:
        # Uroman.romanize_string() is the correct method
        romanized = uroman_obj.romanize_string(text)
        return romanized.strip()
    except Exception as e:
        print(f"Error romanizing text: {text[:50]}... Error: {e}")
        return text # Fallback to original if romanization fails

# Test romanization on a sample
sample_text = train_data[0]['transcription']
print(f"Original (Ge'ez): {sample_text}")
romanized_sample = romanize_text(sample_text)
print(f"Romanized: {romanized_sample}")



## 5. Load Model and Tokenizer


In [ ]:
print(f"Loading model: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = VitsModel.from_pretrained(MODEL_NAME)

print(f"Model loaded successfully!")
print(f"Sampling rate: {model.config.sampling_rate} Hz")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.2f}M")

if torch.cuda.is_available():
 model = model.to("cuda")
 print("Model moved to CUDA")


## 6. Test Inference (Before Training)

Test the model with romanized text to verify it works correctly.


In [ ]:
def synthesize_speech(model, tokenizer, text_romanized, output_path):
    """Synthesize speech from romanized text"""
    inputs = tokenizer(text_romanized, return_tensors="pt")

    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        output = model(**inputs).waveform

    # Move tensor to CPU before converting to numpy
    if isinstance(output, torch.Tensor):
        output = output.cpu()

    # Save audio
    scipy.io.wavfile.write(
        output_path,
        rate=model.config.sampling_rate,
        data=output.numpy().T
    )

    return output_path

# Test with sample text (after romanization)
if len(train_data) > 0 and 'transcription_romanized' in train_data[0]:
    test_text_romanized = train_data[0]['transcription_romanized']
    test_output = "test_synthesized_before_training.wav"

    print(f"Testing synthesis with: {test_text_romanized}")
    synthesize_speech(model, tokenizer, test_text_romanized, test_output)
    print(f"Test audio saved to: {test_output}")
else:
    print("Please run romanization cells first!")




## 7. Apply LoRA (if supported)

**Important Note**: VITS models use a different architecture than transformers. LoRA support may be limited. We'll attempt to apply it, but may need to use standard fine-tuning if LoRA is not compatible.


In [ ]:
# Attempt to apply LoRA
# Note: This may fail for VITS models - if so, we'll need to use standard fine-tuning
try:
    # Check available modules in the model
    print("Checking model structure for LoRA-compatible modules...")
    target_modules_found = []
    for name, module in model.named_modules():
        module_type = type(module).__name__
        if any(target in name.lower() for target in ["linear", "proj", "dense"]):
            if "Linear" in module_type or "Conv" in module_type:
                target_modules_found.append(name)

    print(f"Found {len(target_modules_found)} potential target modules")
    if len(target_modules_found) > 0:
        print("Sample modules:", target_modules_found[:5])

    # Try to configure LoRA with found modules
    # VITS models may not have standard transformer modules
    # We may need to identify the correct target modules
    lora_config = LoraConfig(
        r=LORA_CONFIG["r"],
        lora_alpha=LORA_CONFIG["lora_alpha"],
        target_modules=LORA_CONFIG["target_modules"], # Try standard modules first
        lora_dropout=LORA_CONFIG["lora_dropout"],
        bias=LORA_CONFIG["bias"],
        task_type=TaskType.FEATURE_EXTRACTION
    )

    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    print("\nLoRA adapters applied successfully!")

except Exception as e:
    print(f"LoRA application failed: {e}")
    print("\nNote: VITS models may not support standard LoRA.")
    print("We may need to use standard fine-tuning instead.")
    print("Continuing with standard model for now...")

# Remove any existing wrapper code (safety check)
# If you have any wrapper code, remove it first before running this

# Check if LoRA was successfully applied
from peft import PeftModel

if isinstance(model, PeftModel):
    print("Model is a PeftModel")
    print(f"Base model type: {type(model.base_model)}")
    print(f"Has base_model.forward: {hasattr(model.base_model, 'forward')}")

    # Try to check if we can access the underlying VITS model
    if hasattr(model.base_model, 'model'):
        print(f"Base model has 'model' attribute: {type(model.base_model.model)}")
else:
    print("Model is NOT a PeftModel - LoRA may not have been applied")

# Fix: Patch PEFT's forward to filter out inputs_embeds (VitsModel doesn't accept it)
# This is necessary because PEFT includes inputs_embeds in its forward signature,
# but VitsModel doesn't accept it
if isinstance(model, PeftModel):
    import functools
    
    # Store the original forward method
    original_peft_forward = model.forward
    
    @functools.wraps(original_peft_forward)
    def patched_peft_forward(*args, **kwargs):
        # Remove inputs_embeds if present (VitsModel doesn't accept it)
        kwargs.pop('inputs_embeds', None)
        # Also remove task_ids which PEFT might pass
        kwargs.pop('task_ids', None)
        # Filter to only valid VitsModel arguments
        valid_kwargs = {k: v for k, v in kwargs.items() 
                        if k in ['input_ids', 'attention_mask', 'speaker_id', 
                                'output_attentions', 'output_hidden_states', 'return_dict', 'labels']}
        # Call the original forward with filtered kwargs
        return original_peft_forward(*args, **valid_kwargs)
    
    # Replace the forward method
    model.forward = patched_peft_forward
    
    # Also patch the base_model's forward to catch any internal calls
    if hasattr(model, 'base_model'):
        # Helper function to create a patched forward
        def create_patched_forward(original_forward):
            @functools.wraps(original_forward)
            def patched_forward(*args, **kwargs):
                kwargs.pop('inputs_embeds', None)
                kwargs.pop('task_ids', None)
                valid_kwargs = {k: v for k, v in kwargs.items() 
                                if k in ['input_ids', 'attention_mask', 'speaker_id', 
                                        'output_attentions', 'output_hidden_states', 'return_dict', 'labels']}
                return original_forward(*args, **valid_kwargs)
            return patched_forward
        
        # Recursively patch nested base_models
        current = model.base_model
        depth = 0
        max_depth = 5 # Safety limit
        
        while depth < max_depth:
            if hasattr(current, 'forward'):
                current.forward = create_patched_forward(current.forward)
            
            # Move to next level
            if hasattr(current, 'base_model'):
                current = current.base_model
            elif hasattr(current, 'model'):
                current = current.model
            else:
                break
            depth += 1
        
    print("Patched PEFT model forward to filter out unsupported arguments (inputs_embeds, etc.)")




## 8. Training Setup

**Important**: VITS models require custom training loops as they don't use the standard HuggingFace Trainer interface. This section provides a basic structure. For actual training, you may need to implement a custom training loop.

**Alternative Approach**: If LoRA doesn't work, consider:
1. Standard fine-tuning (full model fine-tuning)
2. Using a different TTS model that supports LoRA better
3. Implementing custom LoRA for VITS architecture


In [ ]:
# Custom training loop for VITS model
# Note: This is a simplified training loop - VITS training is complex
# You may need to adapt this based on your specific requirements

from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.nn as nn

# Prepare datasets for training
train_dataset_list = [
 {'text': item['transcription_romanized'], 'audio_path': item['audio_path']}
 for item in train_data
]

val_dataset_list = [
 {'text': item['transcription_romanized'], 'audio_path': item['audio_path']}
 for item in val_data
]

def load_audio(audio_path, target_sr=16000):
 """Load and resample audio"""
 audio, sr = librosa.load(audio_path, sr=target_sr)
 return audio

print(f"Prepared {len(train_dataset_list)} training samples")
print(f"Prepared {len(val_dataset_list)} validation samples")
print("\nNOTE: Full VITS training requires complex loss functions (mel-spectrogram loss, etc.)")
print("This is a simplified structure. For production use, implement full VITS training objectives.")


In [ ]:
# Memory optimization setup before training
import os

# Set PyTorch CUDA memory allocation config to reduce fragmentation
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear any existing CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    
    # Print current memory usage
    allocated = torch.cuda.memory_allocated(0) / 1e9
    reserved = torch.cuda.memory_reserved(0) / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    
    print("GPU Memory Status:")
    print(f" Allocated: {allocated:.2f} GB")
    print(f" Reserved: {reserved:.2f} GB")
    print(f" Total: {total:.2f} GB")
    print(f" Free: {total - reserved:.2f} GB")
    print()
    
    # Check model memory usage
    if hasattr(model, 'parameters'):
        model_params = sum(p.numel() * p.element_size() for p in model.parameters()) / 1e9
        print(f"Model parameters memory: ~{model_params:.2f} GB (FP32)")
        if TRAINING_ARGS.get("fp16", False):
            print(f" With FP16: ~{model_params / 2:.2f} GB")
        print()




## 9. Train Model

**Note**: This is a simplified training approach. Full VITS training requires complex loss functions. For production, you may need to implement proper VITS training objectives.


In [ ]:
# Initialize tracking lists for training/validation metrics
train_losses = []
val_losses = []
train_steps = []
val_steps = []

print('Initialized metric tracking lists: train_losses, val_losses, train_steps, val_steps')


In [ ]:
# Validation evaluation function
def evaluate_validation(model, tokenizer, val_dataset_list, batch_size=4, use_fp16=False):
    """Run validation evaluation on validation dataset."""
    model.eval()
    total_loss = 0.0
    num_samples = 0
 
    with torch.no_grad():
        for i in range(0, len(val_dataset_list), batch_size):
            batch = val_dataset_list[i:i+batch_size]
 
            try:
                # Tokenize batch text
                texts = [item['text'] for item in batch]
                inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
 
                if torch.cuda.is_available():
                    inputs = {k: v.to("cuda") for k, v in inputs.items()}
 
                # Filter out inputs_embeds which VitsModel doesn't accept
                valid_inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
 
                # Forward pass
                if use_fp16:
                    from torch.cuda.amp import autocast
                    with autocast():
                        outputs = model(**valid_inputs)
                else:
                    outputs = model(**valid_inputs)
 
                # Compute loss (simplified - same as training)
                loss = torch.tensor(0.0, device=inputs['input_ids'].device)
 
                total_loss += loss.item() * len(batch)
                num_samples += len(batch)
 
                del outputs, inputs, valid_inputs
                torch.cuda.empty_cache() if torch.cuda.is_available() else None
 
            except Exception as e:
                print(f"Error in validation batch: {e}")
                continue
 
    model.train()
    avg_val_loss = total_loss / num_samples if num_samples > 0 else 0.0
    return avg_val_loss

print("Validation evaluation function defined.")




In [ ]:
# Simplified training loop with MEMORY OPTIMIZATION
# WARNING: This is a basic structure - full VITS training is more complex

# Enable gradient checkpointing to save memory
if TRAINING_ARGS.get("gradient_checkpointing", False):
    try:
        if hasattr(model, 'gradient_checkpointing_enable'):
            model.gradient_checkpointing_enable()
            print(" Gradient checkpointing enabled")
        elif hasattr(model, 'base_model') and hasattr(model.base_model, 'gradient_checkpointing_enable'):
            model.base_model.gradient_checkpointing_enable()
            print(" Gradient checkpointing enabled (via base_model)")
    except Exception as e:
        print(f"Warning: Could not enable gradient checkpointing: {e}")

# Enable FP16 mixed precision training
use_fp16 = TRAINING_ARGS.get("fp16", False)
if use_fp16:
    from torch.cuda.amp import autocast, GradScaler
    scaler = GradScaler()
    print(" FP16 mixed precision enabled")
else:
    scaler = None

model.train()
optimizer = AdamW(model.parameters(), lr=TRAINING_ARGS["learning_rate"])
scheduler = CosineAnnealingLR(optimizer, T_max=TRAINING_ARGS["max_steps"])

# Reduce batch size if memory is tight (T4 15GB)
batch_size = TRAINING_ARGS["per_device_train_batch_size"]
if batch_size > 2:
    print(f"Warning: Reducing batch size from {batch_size} to 2 for T4 GPU memory constraints")
    batch_size = 2

gradient_accumulation_steps = TRAINING_ARGS["gradient_accumulation_steps"]
max_steps = TRAINING_ARGS["max_steps"]

print("Starting training...")
print(f"Training steps: {max_steps}")
print(f"Learning rate: {TRAINING_ARGS['learning_rate']}")
print(f"Batch size: {batch_size}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Effective batch size: {batch_size * gradient_accumulation_steps}")
print(f"FP16: {use_fp16}")
print()

# Clear CUDA cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"Initial GPU memory: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB / {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print()

step = 0
training_complete = False
epoch = 0
accumulated_loss = 0.0

while step < max_steps:
    epoch_loss = 0.0
    num_batches = 0

    # Simple batching (you may want to implement proper DataLoader)
    for i in tqdm(range(0, len(train_dataset_list), batch_size), desc=f"Epoch {epoch+1}"):
        # CRITICAL: Check step before processing batch to stop at max_steps
        if step >= max_steps:
            training_complete = True
            break

        batch = train_dataset_list[i:i+batch_size]

        # Clear gradients at the start of each batch
        optimizer.zero_grad()

        # Simplified forward pass (actual VITS training needs proper loss)
        try:
            # Tokenize batch text
            texts = [item['text'] for item in batch]
            inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

            if torch.cuda.is_available():
                inputs = {k: v.to("cuda") for k, v in inputs.items()}

            # Forward pass with FP16 if enabled
            # Filter out inputs_embeds which VitsModel doesn't accept
            valid_inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}
 
            # Use autocast for FP16
            if use_fp16:
                with autocast():
                    outputs = model(**valid_inputs)
            else:
                outputs = model(**valid_inputs)

            # PLACEHOLDER: Compute loss (VITS needs complex loss function)
            # For now, we'll skip actual loss computation
            # In real training, you'd compute mel-spectrogram loss, duration loss, etc.

            # Dummy loss for demonstration (REPLACE with actual VITS loss)
            # Use a simple tensor that doesn't require keeping outputs in memory
            loss = torch.tensor(0.0, requires_grad=True, device=inputs['input_ids'].device)

            # Delete outputs immediately to free memory
            del outputs
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

            # Backward pass with FP16 scaling
            if use_fp16:
                scaler.scale(loss).backward()
            else:
                loss.backward()

            # Delete loss and inputs to free memory
            loss_value = loss.item()
            # Track training loss and step (approximate global step)
            train_losses.append(loss_value)
            train_steps.append(step + 1)
            del loss, inputs, valid_inputs
            torch.cuda.empty_cache() if torch.cuda.is_available() else None

            accumulated_loss += loss_value
            num_batches += 1

            # Update weights after gradient accumulation
            if (step + 1) % gradient_accumulation_steps == 0:
                if use_fp16:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
 
                scheduler.step()
                step += 1

                # CRITICAL: Check immediately after incrementing step
                if step >= max_steps:
                    training_complete = True
                    break


                if step % TRAINING_ARGS["logging_steps"] == 0:
                    avg_loss = accumulated_loss / (TRAINING_ARGS["logging_steps"] * gradient_accumulation_steps)
                    mem_used = torch.cuda.memory_allocated(0) / 1e9 if torch.cuda.is_available() else 0
                    mem_total = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
                    print(f"Step {step}/{max_steps} - Loss: {avg_loss:.4f} - GPU Memory: {mem_used:.2f}/{mem_total:.2f} GB")
                    accumulated_loss = 0.0
 
                # Periodic memory cleanup
                torch.cuda.empty_cache()

                # Run actual validation at eval_steps
                if step % TRAINING_ARGS['eval_steps'] == 0:
                    print(f"Running validation at step {step}...")
                    val_loss = evaluate_validation(model, tokenizer, val_dataset_list, batch_size=2, use_fp16=use_fp16)
                    val_losses.append(val_loss)
                    val_steps.append(step)
                    print(f"Validation Loss: {val_loss:.4f}")

                    if step >= max_steps:
                        training_complete = True
                        break

            if step >= max_steps:
                training_complete = True
                break

        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"Warning: CUDA OOM at step {step}. Clearing cache and reducing batch size...")
                torch.cuda.empty_cache()
                # Try with smaller batch
                if batch_size > 1:
                    batch_size = 1
                    print(f"Reduced batch size to {batch_size}")
                else:
                    print("Batch size already at minimum. Skipping this batch.")
                    continue
            else:
                print(f"Error in training step: {e}")
                # Clear memory on any error
                torch.cuda.empty_cache()
                continue
        except Exception as e:
            print(f"Error in training step: {e}")
            # Clear memory on any error
            torch.cuda.empty_cache()
            continue
 
        if training_complete:
            break

    # CRITICAL: Check before starting new epoch
    if training_complete or step >= max_steps:
        break

    epoch += 1

print("\nTraining completed!")

# Final memory cleanup
torch.cuda.empty_cache()

# Save final model
final_model_path = f"{OUTPUT_DIR}_final"
print(f"Saving model to: {final_model_path}")

model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"Model saved successfully to: {final_model_path}")
print("\nNOTE: This was a simplified training loop.")
print("For production use, implement proper VITS training objectives.")




## Training Visualizations

Visualize training progress, validation metrics, and model performance.

**Note**: Since MMS-TTS uses a custom training loop, losses need to be tracked manually during training. 
The visualization code below assumes losses are stored in lists: `train_losses`, `val_losses`, `train_steps`, `val_steps`.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# NOTE: For MMS-TTS custom training loop, you need to track losses manually
# During training, append to these lists:
# train_losses.append(loss_value)
# train_steps.append(step)
# val_losses.append(val_loss_value) # during validation
# val_steps.append(step) # during validation

# If losses weren't tracked, create empty lists
if 'train_losses' not in globals():
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []
    print("Warning: Warning: Losses not tracked during training. Please modify the training loop to track losses.")
    print(" Add: train_losses.append(loss_value) and train_steps.append(step) in the training loop")

# Create comprehensive figure
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Training Loss (top left, spans 2 columns)
ax1 = fig.add_subplot(gs[0, :])
if train_losses and train_steps:
    ax1.plot(train_steps, train_losses, 'b-', linewidth=2.5, label='Training Loss', alpha=0.8)
    ax1.set_xlabel('Training Steps', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(fontsize=12, loc='best')
    ax1.set_facecolor('#f8f9fa')
else:
    ax1.text(0.5, 0.5, 'No training data available\nPlease track losses during training', ha='center', va='center', fontsize=12)
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)

# Plot 2: Validation Loss
ax2 = fig.add_subplot(gs[1, 0])
if val_losses and val_steps:
    ax2.plot(val_steps, val_losses, 'r-', linewidth=2.5, marker='o', markersize=5, label='Validation Loss', alpha=0.8)
    ax2.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.legend(fontsize=11)
    ax2.set_facecolor('#f8f9fa')
else:
    ax2.text(0.5, 0.5, 'No validation data', ha='center', va='center', fontsize=12)
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')

# Plot 3: Combined Loss Plot
ax3 = fig.add_subplot(gs[1, 1])
if train_losses and train_steps:
    ax3.plot(train_steps, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.7)
    if val_losses and val_steps:
        ax3.plot(val_steps, val_losses, 'r-', linewidth=2, marker='o', markersize=4, label='Validation Loss', alpha=0.7)
    ax3.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax3.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.legend(fontsize=11)
    ax3.set_facecolor('#f8f9fa')
else:
    ax3.text(0.5, 0.5, 'No training data', ha='center', va='center', fontsize=12)
    ax3.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')

# Plot 4: Training Statistics Summary
ax4 = fig.add_subplot(gs[2, :])
ax4.axis('off')
if train_losses:
    summary_text = f"Training Summary: {OUTPUT_DIR}\n"
    summary_text += f"Training Loss: Initial={train_losses[0]:.4f}, Final={train_losses[-1]:.4f}, Best={min(train_losses):.4f}\n"
    if val_losses:
        summary_text += f"Validation Loss: Initial={val_losses[0]:.4f}, Final={val_losses[-1]:.4f}, Best={min(val_losses):.4f}"
    ax4.text(0.5, 0.5, summary_text, ha='center', va='center', fontsize=12, family='monospace')
else:
    ax4.text(0.5, 0.5, 'No training statistics available', ha='center', va='center', fontsize=12)

# Add overall title
fig.suptitle(f'Training Progress: {OUTPUT_DIR}', fontsize=18, fontweight='bold', y=0.995)

plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

# Save high-resolution plot
plot_path = f"{OUTPUT_DIR}_training_plots.png"
fig.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\nTraining plots saved to: {plot_path}")




In [ ]:
# Training and Validation Loss Plot (Epoch-based)
import matplotlib.pyplot as plt
import numpy as np

# NOTE: For epoch-based plot, you need to track losses by epoch during training
# Group losses by epoch if available
if train_losses and train_steps:
    # For simplicity, assume each step is roughly an epoch (adjust as needed)
    # In real training, you'd group by actual epoch number
    epochs = list(range(len(train_losses)))
    
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.8)
    if val_losses:
        val_epochs = list(range(len(val_losses)))
        plt.plot(val_epochs, val_losses, 'orange', linewidth=2, label='Validation Loss', alpha=0.8)
    
    plt.xlabel('Epochs', fontsize=12, fontweight='bold')
    plt.ylabel('Loss', fontsize=12, fontweight='bold')
    plt.title('Training and Validation Loss', fontsize=14, fontweight='bold', pad=15)
    plt.legend(fontsize=11, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    plot_path = f"{OUTPUT_DIR}_loss_epochs.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Epoch-based loss plot saved to: {plot_path}")
else:
    print("Warning: No training losses available. Please track losses during training.")




## 10. Zip Model and Copy to Google Drive


In [ ]:
import zipfile
from pathlib import Path

# Zip the final model directory (saved to Kaggle working directory)
zip_filename = f"{Path(final_model_path).name}.zip"
print(f"Creating zip file: {zip_filename}...")

with zipfile.ZipFile(zip_filename, "w", zipfile.ZIP_DEFLATED) as zipf:
    for file_path in Path(final_model_path).rglob("*"):
        if file_path.is_file():
            arcname = file_path.relative_to(final_model_path)
            zipf.write(file_path, arcname)
            print(f" Added: {arcname}")

print(f"\nZip file created: {zip_filename}")
print(f"File size: {Path(zip_filename).stat().st_size / (1024*1024):.2f} MB")
print("\nModel and zip file saved to Kaggle working directory (persistent)")




## 11. Test Inference (After Training)

Test the fine-tuned model with romanized text.


In [ ]:
# Load the fine-tuned model for testing
final_model_path = f"{OUTPUT_DIR}_final"

print(f"Loading fine-tuned model from: {final_model_path}")
test_model = VitsModel.from_pretrained(final_model_path)
test_tokenizer = AutoTokenizer.from_pretrained(final_model_path)

if torch.cuda.is_available():
 test_model = test_model.to("cuda")
 test_model.eval()

# Test with sample text
if len(test_data) > 0 and 'transcription_romanized' in test_data[0]:
 test_text_romanized = test_data[0]['transcription_romanized']
 test_output = "test_synthesized_after_training.wav"

 print(f"\nTesting synthesis with: {test_text_romanized}")
 synthesize_speech(test_model, test_tokenizer, test_text_romanized, test_output)
 print(f"Test audio saved to: {test_output}")
else:
 print("Please run romanization cells first!")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# NOTE: MMS-TTS uses custom training loop, so we use manually tracked losses
# These should be populated during training: train_losses, val_losses, train_steps, val_steps

# Check if losses were tracked
if 'train_losses' not in globals() or not train_losses:
    print("Warning: Training losses not tracked. Please ensure the training loop tracks losses.")
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []

# For WER and CER, these are not tracked in MMS-TTS custom loop
# Set to empty lists (they would need to be added to the training loop if needed)
eval_wers = []
eval_cers = []

# Create comprehensive figure
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Training Loss (top left, spans 2 columns)
ax1 = fig.add_subplot(gs[0, :])
if train_losses and train_steps:
    ax1.plot(train_steps, train_losses, 'b-', linewidth=2.5, label='Training Loss', alpha=0.8)
    ax1.set_xlabel('Training Steps', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(fontsize=12, loc='best')
    ax1.set_facecolor('#f8f9fa')
else:
    ax1.text(0.5, 0.5, 'No training data available\nPlease track losses during training', ha='center', va='center', fontsize=12)
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)

# Plot 2: Validation Loss
ax2 = fig.add_subplot(gs[1, 0])
if val_losses and val_steps:
    ax2.plot(val_steps, val_losses, 'r-', linewidth=2.5, marker='o', markersize=5, label='Validation Loss', alpha=0.8)
    ax2.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.legend(fontsize=11)
    ax2.set_facecolor('#f8f9fa')
else:
    ax2.text(0.5, 0.5, 'No validation data', ha='center', va='center', fontsize=12)
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')

# Plot 3: Word Error Rate (WER) - Not available for MMS-TTS
ax3 = fig.add_subplot(gs[1, 1])
if eval_wers and any(w > 0 for w in eval_wers):
    ax3.plot(val_steps, eval_wers, 'g-', linewidth=2.5, marker='s', markersize=5, label='WER', alpha=0.8)
    ax3.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Word Error Rate', fontsize=12, fontweight='bold')
    ax3.set_title('Word Error Rate (WER)', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.legend(fontsize=11)
    ax3.set_ylim(bottom=0)
    ax3.set_facecolor('#f8f9fa')
else:
    ax3.text(0.5, 0.5, 'WER not tracked\n(Not available for MMS-TTS)', ha='center', va='center', fontsize=12)
    ax3.set_title('Word Error Rate (WER)', fontsize=14, fontweight='bold')

# Plot 4: Character Error Rate (CER) - Not available for MMS-TTS
ax4 = fig.add_subplot(gs[2, 0])
if eval_cers and any(c > 0 for c in eval_cers):
    ax4.plot(val_steps, eval_cers, 'm-', linewidth=2.5, marker='^', markersize=5, label='CER', alpha=0.8)
    ax4.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Character Error Rate', fontsize=12, fontweight='bold')
    ax4.set_title('Character Error Rate (CER)', fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3, linestyle='--')
    ax4.legend(fontsize=11)
    ax4.set_ylim(bottom=0)
    ax4.set_facecolor('#f8f9fa')
else:
    ax4.text(0.5, 0.5, 'CER not tracked\n(Not available for MMS-TTS)', ha='center', va='center', fontsize=12)
    ax4.set_title('Character Error Rate (CER)', fontsize=14, fontweight='bold')

# Plot 5: Combined Loss Plot (Training vs Validation)
ax5 = fig.add_subplot(gs[2, 1])
if train_losses and train_steps:
    ax5.plot(train_steps, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.7)
    if val_losses and val_steps:
        ax5.plot(val_steps, val_losses, 'r-', linewidth=2, marker='o', markersize=4, label='Validation Loss', alpha=0.7)
    ax5.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax5.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
    ax5.grid(True, alpha=0.3, linestyle='--')
    ax5.legend(fontsize=11)
    ax5.set_facecolor('#f8f9fa')
else:
    ax5.text(0.5, 0.5, 'No training data', ha='center', va='center', fontsize=12)
    ax5.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')

# Add overall title
fig.suptitle(f'Training Progress: {OUTPUT_DIR}', fontsize=18, fontweight='bold', y=0.995)

plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

# Save high-resolution plot
plot_path = f"{OUTPUT_DIR}_training_plots.png"
fig.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\nTraining plots saved to: {plot_path}")

# Print detailed summary statistics
print("\n" + "="*70)
print(f"TRAINING SUMMARY: {OUTPUT_DIR}")
print("="*70)
if train_losses:
    print(f"\nTraining Loss:")
    print(f" Initial: {train_losses[0]:.4f}")
    print(f" Final: {train_losses[-1]:.4f}")
    print(f" Best: {min(train_losses):.4f} (at step {train_steps[train_losses.index(min(train_losses))]})")
    print(f" Improvement: {((train_losses[0] - min(train_losses)) / train_losses[0] * 100):.2f}%")
if val_losses:
    print(f"\nValidation Loss:")
    print(f" Initial: {val_losses[0]:.4f}")
    print(f" Final: {val_losses[-1]:.4f}")
    print(f" Best: {min(val_losses):.4f} (at step {val_steps[val_losses.index(min(val_losses))]})")
    print(f" Improvement: {((val_losses[0] - min(val_losses)) / val_losses[0] * 100):.2f}%")
if eval_wers and any(w > 0 for w in eval_wers):
    valid_wers = [w for w in eval_wers if w > 0]
    valid_steps = [val_steps[i] for i, w in enumerate(eval_wers) if w > 0]
    print(f"\nWord Error Rate (WER):")
    print(f" Initial: {valid_wers[0]:.4f}")
    print(f" Final: {valid_wers[-1]:.4f}")
    print(f" Best: {min(valid_wers):.4f} (at step {valid_steps[valid_wers.index(min(valid_wers))]})")
    print(f" Improvement: {((valid_wers[0] - min(valid_wers)) / valid_wers[0] * 100):.2f}%")
if eval_cers and any(c > 0 for c in eval_cers):
    valid_cers = [c for c in eval_cers if c > 0]
    valid_steps = [val_steps[i] for i, c in enumerate(eval_cers) if c > 0]
    print(f"\nCharacter Error Rate (CER):")
    print(f" Initial: {valid_cers[0]:.4f}")
    print(f" Final: {valid_cers[-1]:.4f}")
    print(f" Best: {min(valid_cers):.4f} (at step {valid_steps[valid_cers.index(min(valid_cers))]})")
    print(f" Improvement: {((valid_cers[0] - min(valid_cers)) / valid_cers[0] * 100):.2f}%")
print("="*70)



In [ ]:
# Training and Validation Loss Plot (Epoch-based)
import matplotlib.pyplot as plt
import numpy as np

# NOTE: MMS-TTS uses custom training loop, so we use manually tracked losses
# These should be populated during training: train_losses, val_losses, train_steps, val_steps

# Check if losses were tracked
if 'train_losses' not in globals() or not train_losses:
    print("Warning: Training losses not tracked. Please ensure the training loop tracks losses.")
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []

# For epoch-based plot, we'll use steps as approximate epochs
# In a real scenario, you'd track actual epoch numbers during training
if train_losses and train_steps:
    # For simplicity, assume each step is roughly an epoch (adjust as needed)
    # In real training, you'd group by actual epoch number
    epochs = list(range(len(train_losses)))
    
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.8)
    if val_losses:
        val_epochs = list(range(len(val_losses)))
        plt.plot(val_epochs, val_losses, 'orange', linewidth=2, label='Validation Loss', alpha=0.8)
    
    plt.xlabel('Epochs', fontsize=12, fontweight='bold')
    plt.ylabel('Loss', fontsize=12, fontweight='bold')
    plt.title('Training and Validation Loss', fontsize=14, fontweight='bold', pad=15)
    plt.legend(fontsize=11, loc='best')
    plt.grid(True, alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    plot_path = f"{OUTPUT_DIR}_loss_epochs.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Epoch-based loss plot saved to: {plot_path}")
else:
    print("Warning: No training losses available. Please track losses during training.")



# Training and Validation Loss Plot (Epoch-based)
import matplotlib.pyplot as plt
import numpy as np

# Extract training history from trainer state
train_history = trainer.state.log_history

# Separate training and evaluation logs
train_logs = [log for log in train_history if 'loss' in log and 'eval_loss' not in log]
eval_logs = [log for log in train_history if 'eval_loss' in log]

# Extract epochs and losses
# Group training logs by epoch
train_by_epoch = {}
for log in train_logs:
    epoch = log.get('epoch', 0)
    if epoch not in train_by_epoch:
        train_by_epoch[epoch] = []
    train_by_epoch[epoch].append(log.get('loss', 0))

# Average losses per epoch
epochs = sorted(train_by_epoch.keys())
train_losses = [np.mean(train_by_epoch[epoch]) for epoch in epochs]

# Extract validation losses by epoch
eval_by_epoch = {}
for log in eval_logs:
    epoch = log.get('epoch', 0)
    if epoch not in eval_by_epoch:
        eval_by_epoch[epoch] = []
    eval_by_epoch[epoch].append(log.get('eval_loss', 0))

# Average validation losses per epoch
eval_epochs = sorted(eval_by_epoch.keys())
eval_losses = [np.mean(eval_by_epoch[epoch]) for epoch in eval_epochs]

# Create the plot matching the image style
plt.figure(figsize=(10, 6))
plt.plot(epochs, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.8)
if eval_epochs:
    plt.plot(eval_epochs, eval_losses, 'orange', linewidth=2, label='Validation Loss', alpha=0.8)

plt.xlabel('Epochs', fontsize=12, fontweight='bold')
plt.ylabel('Loss', fontsize=12, fontweight='bold')
plt.title('Training and Validation Loss', fontsize=14, fontweight='bold', pad=15)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

# Save the plot
plot_path = f"{OUTPUT_DIR}_loss_epochs.png"
plt.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"Epoch-based loss plot saved to: {plot_path}")




In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# NOTE: MMS-TTS uses custom training loop, so we use manually tracked losses
# These should be populated during training: train_losses, val_losses, train_steps, val_steps

# Check if losses were tracked
if 'train_losses' not in globals() or not train_losses:
    print("Warning: Training losses not tracked. Please ensure the training loop tracks losses.")
    train_losses = []
    train_steps = []
    val_losses = []
    val_steps = []

# For WER and CER, these are not tracked in MMS-TTS custom loop
# Set to empty lists (they would need to be added to the training loop if needed)
eval_wers = []
eval_cers = []

# Create comprehensive figure
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Training Loss (top left, spans 2 columns)
ax1 = fig.add_subplot(gs[0, :])
if train_losses and train_steps:
    ax1.plot(train_steps, train_losses, 'b-', linewidth=2.5, label='Training Loss', alpha=0.8)
    ax1.set_xlabel('Training Steps', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)
    ax1.grid(True, alpha=0.3, linestyle='--')
    ax1.legend(fontsize=12, loc='best')
    ax1.set_facecolor('#f8f9fa')
else:
    ax1.text(0.5, 0.5, 'No training data available\nPlease track losses during training', ha='center', va='center', fontsize=12)
    ax1.set_title('Training Loss Over Time', fontsize=15, fontweight='bold', pad=15)

# Plot 2: Validation Loss
ax2 = fig.add_subplot(gs[1, 0])
if val_losses and val_steps:
    ax2.plot(val_steps, val_losses, 'r-', linewidth=2.5, marker='o', markersize=5, label='Validation Loss', alpha=0.8)
    ax2.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, linestyle='--')
    ax2.legend(fontsize=11)
    ax2.set_facecolor('#f8f9fa')
else:
    ax2.text(0.5, 0.5, 'No validation data', ha='center', va='center', fontsize=12)
    ax2.set_title('Validation Loss', fontsize=14, fontweight='bold')

# Plot 3: Word Error Rate (WER) - Not available for MMS-TTS
ax3 = fig.add_subplot(gs[1, 1])
if eval_wers and any(w > 0 for w in eval_wers):
    ax3.plot(val_steps, eval_wers, 'g-', linewidth=2.5, marker='s', markersize=5, label='WER', alpha=0.8)
    ax3.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Word Error Rate', fontsize=12, fontweight='bold')
    ax3.set_title('Word Error Rate (WER)', fontsize=14, fontweight='bold')
    ax3.grid(True, alpha=0.3, linestyle='--')
    ax3.legend(fontsize=11)
    ax3.set_ylim(bottom=0)
    ax3.set_facecolor('#f8f9fa')
else:
    ax3.text(0.5, 0.5, 'WER not tracked\n(Not available for MMS-TTS)', ha='center', va='center', fontsize=12)
    ax3.set_title('Word Error Rate (WER)', fontsize=14, fontweight='bold')

# Plot 4: Character Error Rate (CER) - Not available for MMS-TTS
ax4 = fig.add_subplot(gs[2, 0])
if eval_cers and any(c > 0 for c in eval_cers):
    ax4.plot(val_steps, eval_cers, 'm-', linewidth=2.5, marker='^', markersize=5, label='CER', alpha=0.8)
    ax4.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Character Error Rate', fontsize=12, fontweight='bold')
    ax4.set_title('Character Error Rate (CER)', fontsize=14, fontweight='bold')
    ax4.grid(True, alpha=0.3, linestyle='--')
    ax4.legend(fontsize=11)
    ax4.set_ylim(bottom=0)
    ax4.set_facecolor('#f8f9fa')
else:
    ax4.text(0.5, 0.5, 'CER not tracked\n(Not available for MMS-TTS)', ha='center', va='center', fontsize=12)
    ax4.set_title('Character Error Rate (CER)', fontsize=14, fontweight='bold')

# Plot 5: Combined Loss Plot (Training vs Validation)
ax5 = fig.add_subplot(gs[2, 1])
if train_losses and train_steps:
    ax5.plot(train_steps, train_losses, 'b-', linewidth=2, label='Training Loss', alpha=0.7)
    if val_losses and val_steps:
        ax5.plot(val_steps, val_losses, 'r-', linewidth=2, marker='o', markersize=4, label='Validation Loss', alpha=0.7)
    ax5.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax5.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax5.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')
    ax5.grid(True, alpha=0.3, linestyle='--')
    ax5.legend(fontsize=11)
    ax5.set_facecolor('#f8f9fa')
else:
    ax5.text(0.5, 0.5, 'No training data', ha='center', va='center', fontsize=12)
    ax5.set_title('Training vs Validation Loss', fontsize=14, fontweight='bold')

# Add overall title
fig.suptitle(f'Training Progress: {OUTPUT_DIR}', fontsize=18, fontweight='bold', y=0.995)

plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

# Save high-resolution plot
plot_path = f"{OUTPUT_DIR}_training_plots.png"
fig.savefig(plot_path, dpi=300, bbox_inches='tight', facecolor='white')
print(f"\nTraining plots saved to: {plot_path}")

# Print detailed summary statistics
print("\n" + "="*70)
print(f"TRAINING SUMMARY: {OUTPUT_DIR}")
print("="*70)
if train_losses:
    print(f"\nTraining Loss:")
    print(f" Initial: {train_losses[0]:.4f}")
    print(f" Final: {train_losses[-1]:.4f}")
    print(f" Best: {min(train_losses):.4f} (at step {train_steps[train_losses.index(min(train_losses))]})")
    print(f" Improvement: {((train_losses[0] - min(train_losses)) / train_losses[0] * 100):.2f}%")
if val_losses:
    print(f"\nValidation Loss:")
    print(f" Initial: {val_losses[0]:.4f}")
    print(f" Final: {val_losses[-1]:.4f}")
    print(f" Best: {min(val_losses):.4f} (at step {val_steps[val_losses.index(min(val_losses))]})")
    print(f" Improvement: {((val_losses[0] - min(val_losses)) / val_losses[0] * 100):.2f}%")
if eval_wers and any(w > 0 for w in eval_wers):
    valid_wers = [w for w in eval_wers if w > 0]
    valid_steps = [val_steps[i] for i, w in enumerate(eval_wers) if w > 0]
    print(f"\nWord Error Rate (WER):")
    print(f" Initial: {valid_wers[0]:.4f}")
    print(f" Final: {valid_wers[-1]:.4f}")
    print(f" Best: {min(valid_wers):.4f} (at step {valid_steps[valid_wers.index(min(valid_wers))]})")
    print(f" Improvement: {((valid_wers[0] - min(valid_wers)) / valid_wers[0] * 100):.2f}%")
if eval_cers and any(c > 0 for c in eval_cers):
    valid_cers = [c for c in eval_cers if c > 0]
    valid_steps = [val_steps[i] for i, c in enumerate(eval_cers) if c > 0]
    print(f"\nCharacter Error Rate (CER):")
    print(f" Initial: {valid_cers[0]:.4f}")
    print(f" Final: {valid_cers[-1]:.4f}")
    print(f" Best: {min(valid_cers):.4f} (at step {valid_steps[valid_cers.index(min(valid_cers))]})")
    print(f" Improvement: {((valid_cers[0] - min(valid_cers)) / valid_cers[0] * 100):.2f}%")
print("="*70)

